In [1]:
import mysql.connector
from getpass import getpass
import pandas as pd

In [2]:
conn = mysql.connector.connect(host = "localhost",
                               user= "root",
                               password=getpass())
# now are creating a curseor, that will hold our query, execute it, and fetch results
cursor = conn.cursor()

In [3]:
conn.is_connected()

True

In [9]:
def rentals_month(month,year):
    query=  f"""
            select *
            from sakila.rental
            where month(rental_date) = {month} and year(rental_date) = {year}
            """
    cursor.execute(query)
    results = cursor.fetchall()
    data = pd.DataFrame(results, columns= cursor.column_names)
    return data

In [10]:
rentals_month(2,2006)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,11496,2006-02-14 15:16:03,2047,155,None,1,2006-02-15 21:30:53
1,11541,2006-02-14 15:16:03,2026,335,None,1,2006-02-15 21:30:53
2,11563,2006-02-14 15:16:03,1545,83,None,1,2006-02-15 21:30:53
3,11577,2006-02-14 15:16:03,4106,219,None,2,2006-02-15 21:30:53
4,11593,2006-02-14 15:16:03,817,99,None,1,2006-02-15 21:30:53
...,...,...,...,...,...,...,...
177,15862,2006-02-14 15:16:03,925,215,None,1,2006-02-15 21:30:53
178,15867,2006-02-14 15:16:03,837,505,None,2,2006-02-15 21:30:53
179,15875,2006-02-14 15:16:03,3611,41,None,1,2006-02-15 21:30:53
180,15894,2006-02-14 15:16:03,4416,168,None,1,2006-02-15 21:30:53


In [14]:
def rental_count_month(month,year):
    query=  f"""
            select customer_id, count(rental_date) as rentals_{month}_{year}
            from sakila.rental
            where month(rental_date) = {month} and year(rental_date) = {year}
                group by customer_id;
            """
    cursor.execute(query)
    results = cursor.fetchall()
    data = pd.DataFrame(results, columns= cursor.column_names)
    return data


In [53]:
rental_count_month(6,2005)

,customer_id,rentals_6_2005
0,1,7
1,2,1
2,3,4
3,4,6
4,5,5
...,...,...
585,595,2
586,596,2
587,597,3
588,598,1


In [54]:
def compare_rentals(month1,year1,month2,year2):
    df1 = rental_count_month(month1,year1)
    df2 = rental_count_month(month2,year2)
    df3 = df1.merge(df2)
    df3["difference"] = df3.iloc[ : , 2] - df3.iloc[ : , 1] 
    return df3

In [57]:
compare_rentals(5,2005,6,2005)

,customer_id,rentals_5_2005,rentals_6_2005,difference
0,1,2,7,5
1,2,1,1,0
2,3,2,4,2
3,5,3,5,2
4,6,3,4,1
...,...,...,...,...
507,594,4,6,2
508,595,1,2,1
509,596,6,2,-4
510,597,2,3,1
